<a href="https://colab.research.google.com/github/anmolpandeybtech/Chatting-Application/blob/main/Skin_disease(EfficientNet_B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d ascanipek/skin-diseases

100% 6.18G/6.18G [00:39<00:00, 247MB/s]
100% 6.18G/6.18G [00:39<00:00, 166MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/skin-diseases.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 5.6 MB/s eta 0:00:00


In [5]:
from keras import backend

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [7]:
tf.random.set_seed(42)

In [8]:
train_dir = '/content/kaggle/train'
validation_dir = '/content/kaggle/val'
test_dir = '/content/kaggle/test'

target_size = (160, 160)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

batch_size = 16


In [9]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 30909 images belonging to 6 classes.
Found 3923 images belonging to 6 classes.
Found 3928 images belonging to 6 classes.


In [10]:
base_model = EfficientNetB0(input_shape=(target_size[0], target_size[1], 3), include_top=False, weights='imagenet')

NUM_CLASSES = len(train_generator.class_indices)
print(NUM_CLASSES)
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

16705208/16705208 [==============================] - 0s 0us/step
6


In [11]:
initial_learning_rate = 1e-3
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=1000, decay_rate=0.9, staircase=True
)

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=lr_schedule),
    metrics=['accuracy']
)

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 5, 5, 1280)        4049571   
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               163968    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                                 
Total params: 4214313 (16.08 MB)
Trainable params: 41722

In [13]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('skin_disease_model.h5', save_best_only=True)

epochs = 30
history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/30
1931/1931 [==============================] - ETA: 0s - loss: 0.9791 - accuracy: 0.6168

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1931/1931 [==============================] - 353s 157ms/step - loss: 0.9791 - accuracy: 0.6168 - val_loss: 3.1539 - val_accuracy: 0.1347
Epoch 2/30
1931/1931 [==============================] - 270s 140ms/step - loss: 0.7532 - accuracy: 0.7167 - val_loss: 2.6944 - val_accuracy: 0.0742
Epoch 3/30
1931/1931 [==============================] - 272s 141ms/step - loss: 0.6191 - accuracy: 0.7706 - val_loss: 51.6222 - val_accuracy: 0.1804
Epoch 4/30
1931/1931 [==============================] - 292s 151ms/step - loss: 0.4810 - accuracy: 0.8249 - val_loss: 4.1194 - val_accuracy: 0.1301
Epoch 5/30
1931/1931 [==============================] - 295s 153ms/step - loss: 0.3456 - accuracy: 0.8768 - val_loss: 2.8916 - val_accuracy: 0.2064
Epoch 6/30
1931/1931 [==============================] - 280s 145ms/step - loss: 0.2340 - accuracy: 0.9207 - val_loss: 3.8554 - val_accuracy: 0.1250
Epoch 7/30
1931/1931 [==============================] - 280s 145ms/step - loss: 0.1634 - accuracy: 0.9439 - val_loss: 4.32

In [14]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)

245/245 [==============================] - 29s 116ms/step - loss: 2.6753 - accuracy: 0.0847
